In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
folder = '../data/raw_data/'

df_categories = pd.read_csv(f'{folder}df_categories_ai.csv')
df_comments = pd.read_csv(f'{folder}df_comments_video_ai.csv', engine='python', on_bad_lines='skip')
df_transcript = pd.read_csv(f'{folder}df_transcript_original_ai.csv')
df_videos = pd.read_csv(f'{folder}df_video_ai.csv')
df_search = pd.read_csv(f'{folder}df_video_search_ids_ai.csv')
df_channel = pd.read_csv(f'{folder}df_channel_data_ai.csv')

# Clean df_channel_data

In [31]:
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   channel_id        2010 non-null   object
 1   title             2010 non-null   object
 2   description       1857 non-null   object
 3   published_at      2010 non-null   object
 4   subscriber_count  2010 non-null   int64 
 5   video_count       2010 non-null   int64 
 6   view_count        2010 non-null   int64 
 7   region            1583 non-null   object
dtypes: int64(3), object(5)
memory usage: 125.8+ KB


In [32]:
# Drop wrong fetch data 
df_channel.dropna(subset=['view_count'], inplace=True)

# Change to datetime
df_channel['published_at'] = pd.to_datetime(df_channel['published_at'], format='mixed')

# Export cleaned data
df_channel.to_csv('../data/clean_data/df_channel_data.csv', index=False)

# Show info cleaned
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   channel_id        2010 non-null   object             
 1   title             2010 non-null   object             
 2   description       1857 non-null   object             
 3   published_at      2010 non-null   datetime64[ns, UTC]
 4   subscriber_count  2010 non-null   int64              
 5   video_count       2010 non-null   int64              
 6   view_count        2010 non-null   int64              
 7   region            1583 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(3), object(4)
memory usage: 125.8+ KB


# Clean df_video_data

In [33]:
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2711 entries, 0 to 2710
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   videoId         2711 non-null   object
 1   title           2711 non-null   object
 2   channelId       2711 non-null   object
 3   description     2535 non-null   object
 4   publishedAt     2711 non-null   object
 5   thumbnail_url   2711 non-null   object
 6   tags            2711 non-null   object
 7   live_broadcast  2711 non-null   object
 8   categoryId      2711 non-null   int64 
 9   viewCount       2711 non-null   int64 
 10  likeCount       2711 non-null   int64 
 11  commentCount    2711 non-null   int64 
 12  licensed        2711 non-null   bool  
 13  duration        2711 non-null   object
 14  caption         2711 non-null   bool  
 15  language        2711 non-null   object
dtypes: bool(2), int64(4), object(10)
memory usage: 301.9+ KB


In [34]:
import isodate

# Remove duplicated video 
df_videos.drop_duplicates(subset=['videoId'], keep='first', inplace=True)

# Convert ISO 8601 duration to a readable format (timedelta)
def convert_duration(duration_str):
    duration = isodate.parse_duration(duration_str)
    return duration.seconds

# Change duration to total seconds
df_videos['duration'] = df_videos['duration'].apply(convert_duration)

# Drop live broadcast column and Unnamed 0
df_videos.drop(columns=['live_broadcast'], inplace=True)

# Export the cleaned data
df_videos.to_csv('../data/clean_data/df_video_data.csv', index=False)

df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2711 entries, 0 to 2710
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   videoId        2711 non-null   object
 1   title          2711 non-null   object
 2   channelId      2711 non-null   object
 3   description    2535 non-null   object
 4   publishedAt    2711 non-null   object
 5   thumbnail_url  2711 non-null   object
 6   tags           2711 non-null   object
 7   categoryId     2711 non-null   int64 
 8   viewCount      2711 non-null   int64 
 9   likeCount      2711 non-null   int64 
 10  commentCount   2711 non-null   int64 
 11  licensed       2711 non-null   bool  
 12  duration       2711 non-null   int64 
 13  caption        2711 non-null   bool  
 14  language       2711 non-null   object
dtypes: bool(2), int64(5), object(8)
memory usage: 280.8+ KB


# Clean df_comments_video

In [35]:
df_comments.info()
pd.isnull(df_comments).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185382 entries, 0 to 185381
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   comment_id         185382 non-null  object 
 1   author             185003 non-null  object 
 2   author_channel_id  185109 non-null  object 
 3   text               185109 non-null  object 
 4   like_count         184944 non-null  float64
 5   published_at       184944 non-null  object 
 6   updated_at         184779 non-null  object 
 7   totalReplyCount    184779 non-null  float64
 8   video_id           184779 non-null  object 
dtypes: float64(2), object(7)
memory usage: 12.7+ MB


comment_id             0
author               379
author_channel_id    273
text                 273
like_count           438
published_at         438
updated_at           603
totalReplyCount      603
video_id             603
dtype: int64

In [36]:
# Drop na values
df_comments.dropna(inplace=True)

# Transform data to correct type
df_comments['like_count'] = df_comments['like_count'].astype('float64')
df_comments['published_at'] = pd.to_datetime(df_comments['published_at'])
df_comments['updated_at'] = pd.to_datetime(df_comments['updated_at'])

# Export cleaned data
df_comments.to_parquet('../data/clean_data/df_comments_video.parquet', index=False)

df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184673 entries, 0 to 185381
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   comment_id         184673 non-null  object             
 1   author             184673 non-null  object             
 2   author_channel_id  184673 non-null  object             
 3   text               184673 non-null  object             
 4   like_count         184673 non-null  float64            
 5   published_at       184673 non-null  datetime64[ns, UTC]
 6   updated_at         184673 non-null  datetime64[ns, UTC]
 7   totalReplyCount    184673 non-null  float64            
 8   video_id           184673 non-null  object             
dtypes: datetime64[ns, UTC](2), float64(2), object(5)
memory usage: 14.1+ MB


In [37]:
df_comments = pd.read_parquet('../data/clean_data/df_comments_video.parquet')

# Clean df_transcript

In [38]:
df_transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           1787 non-null   object
 1   original_language  1787 non-null   object
 2   transcription      1787 non-null   object
dtypes: object(3)
memory usage: 42.0+ KB


In [39]:
# Drop duplicated values
df_transcript.drop_duplicates(subset=['video_id'], inplace=True)

# Cleaned language values
def get_cleaned_name(str):
    return str.split(' ')[0].strip().lower()

df_transcript['language'] = df_transcript['original_language'].apply(get_cleaned_name)

# Export dataframe
df_transcript.to_csv('../data/clean_data/df_transcript_original.csv')

df_transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1787 entries, 0 to 1786
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           1787 non-null   object
 1   original_language  1787 non-null   object
 2   transcription      1787 non-null   object
 3   language           1787 non-null   object
dtypes: object(4)
memory usage: 56.0+ KB


# Normalize text by language

In [40]:
from langdetect import detect, DetectorFactory, LangDetectException
from nltk.corpus import stopwords

# Create a column with language detected
def detect_language(text):
    try:
        # Verificar si el texto es lo suficientemente largo
        if len(text.strip()) == 0:
            return None
        # Intentar detectar el idioma
        language = detect(text)
        return language
    except LangDetectException:
        # Si el texto es demasiado corto o no se puede detectar el idioma, devolver None
        return None

In [41]:
# Remove specific stopwords from each langua
def remove_stopwords(text, language):
    stop_words = set(stopwords.words(language))
    return ' '.join([word for word in text.split() if word not in stop_words])

df_transcript['transcription_normalized'] = df_transcript[df_transcript['language'] == 'english'].apply(lambda x: remove_stopwords(x['transcription'], x['language']), axis=1)

In [42]:
df_transcript['transcription_normalized']

0       [Music] thank hello everyone I hope great era ...
1       number three [Music] Facebook enacted emergenc...
2       series we're going introduce deep learning lea...
3       earn money AI part two let go want create kind...
4       current Medical Science run options doctors kn...
                              ...                        
1782    machine learning big data changing econometric...
1783    [Music] [Applause] thanks much Hadley thank co...
1784                                                  NaN
1785    happened last year made urgent long time I wor...
1786    there's saying goes great power comes great re...
Name: transcription_normalized, Length: 1787, dtype: object

# Translate Comments to English

In [43]:
df_comments.head()

,comment_id,author,author_channel_id,text,like_count,published_at,updated_at,totalReplyCount,video_id
0,UgxqJ2eO2p7w2NEvB2p4AaABAg,@emmanuelamama1991,UCdUooL3DTt3Wj0M3xRnAC3Q,"great video,helpful.",1.0,2024-08-17 13:07:48+00:00,2024-08-17 13:07:48+00:00,1.0,qtlUwwtvuEg
1,UgzmXwaNzivNHqjfR5l4AaABAg,@dr.alexshayo6972,UCN56M0YoTn18rd_KBFL0QGg,Thank you for such well explained video tutorial,4.0,2023-05-01 14:48:21+00:00,2023-05-01 14:48:21+00:00,1.0,qtlUwwtvuEg
2,UgyySzt4xOZ1hKdS97x4AaABAg,@mohdkashif4596,UCr5Ua5AsmM6phaRu3dBU1qg,"very useful tools , thanks for sharing this in...",2.0,2023-07-20 11:33:23+00:00,2023-07-20 11:33:23+00:00,1.0,qtlUwwtvuEg
3,UgyHVo-IOZw_-uLb-wh4AaABAg,@pipedrmmr,UC5KmDvJCDtoM31gjpGcdYLQ,This is great information and a terrific use o...,13.0,2023-06-25 00:36:50+00:00,2023-06-25 00:36:50+00:00,1.0,qtlUwwtvuEg
4,UgyIT65ucXCc-4loBEx4AaABAg,@anamnaz2527,UCriCWejgmnsDkaX77-Ay8Hw,first two are not goof but all other AI id the...,1.0,2023-05-31 16:41:21+00:00,2023-05-31 16:41:21+00:00,1.0,qtlUwwtvuEg


In [44]:
from langdetect import detect, DetectorFactory, LangDetectException
from nltk.corpus import stopwords

# Create a column with language detected
def detect_language(text):
    try:
        # Verificar si el texto es lo suficientemente largo
        if len(text.strip()) == 0:
            return None
        # Intentar detectar el idioma
        language = detect(text)
        return language
    except LangDetectException:
        # Si el texto es demasiado corto o no se puede detectar el idioma, devolver None
        return None

In [45]:
from deep_translator import GoogleTranslator

def translate_text(text, target_language='en'):
    translator = GoogleTranslator(source='auto', target=target_language)
    return translator.translate(text)

# Detect language

df_test = df_comments[:1000].copy()
df_test['language'] = df_test['text'].apply(detect_language)

In [46]:
df_test['language'][997]

'en'

In [47]:
language = detect(df_test['text'][10])
language

'af'

In [ ]:
df_test['text'][20:25]

20                    Number 3❤❤❤
21                  Moto Rizer...
22                            1+👌
23    unihertz lune smartphone 8g
24              All very nice 👍👍👍
Name: text, dtype: object